Install the necessary dependencies

In [ ]:
!pip install torch==1.13.1 torchdata==0.5.1 transformers==4.27.2 datasets==2.11.0 --quiet

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,GenerationConfig
from datasets import load_dataset

Loading the data from a hugging face dataset

In [ ]:
#loading the data from DialogSum dataset 
hg_dataset_name="knkarthick/dialogsum" 
dataset=load_dataset(hg_dataset_name)

#to print a sample 
print(dataset['test'][0]['dialogue']) 
print('-'.join('' for x in range(200))) 
print(dataset['test'][0]['summary'])

Load the LLM for Text Summarization : Using "google/flan-t5-base" model.
T5: Text-To-Text Transfer Transformer model.

Load the model and tokenizer from transformers

In [ ]:
#load the seq2seq model and tokenizer 
model_name='google/flan-t5-base' 
model=AutoModelForSeq2SeqLM.from_pretrained(model_name) 
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [ ]:
#testing the tokenizer and its working 
sentence= "What day is it, Jerry?" 
sentence_encoded=tokenizer(sentence,return_tensors='pt') 
sentence_decoded=tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True) 
print('ENCODED SENTENCE:',sentence_encoded['input_ids'][0]) 
print("\nDECODED SENTENCE:",sentence_decoded)


In [ ]:
#MODEL GENERATED SUMMARY - WITHOUT PROMPT ENGINEERING

sample_index=[20,60] 
for i,index in enumerate(sample_index): 
    dialogue=dataset['test'][index]['dialogue'] 
    summary=dataset['test'][index]['summary'] 
    inputs=tokenizer(dialogue,return_tensors='pt') 
    output=tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True) 
    
    print('-'.join('' for x in range(100))) 
    print("Example ",i+1) print('-'.join('' for x in range(100))) 
    print(f"INPUT PROMPT:\n{dialogue}") print('-'.join('' for x in range(100))) 
    print(f"BASELINE HUMAN SUMMARY:\n{summary}") 
    print('-'.join('' for x in range(100))) 
    print(f"MODEL GENERATED SUMMARY - WITHOUT PROMPT ENGINEERING:\n{output}\n")


In [ ]:
#MODEL GENERATED SUMMARY - ZERO SHOT
#prompt1
sample_index=[20,60] 
for i,index in enumerate(sample_index): 
    dialogue=dataset['test'][index]['dialogue'] 
    summary=dataset['test'][index]['summary'] 
    prompt=f""" Give an accurate summary of the following conversation with exact reasons from the plot. {dialogue} Summary: """ 
    inputs=tokenizer(prompt,return_tensors='pt') 
    output=tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True) 
    
    print('-'.join('' for x in range(100))) 
    print("Example ",i+1) 
    print('-'.join('' for x in range(100))) 
    print(f"INPUT PROMPT:\n{prompt}") 
    print('-'.join('' for x in range(100))) 
    print(f"BASELINE HUMAN SUMMARY:\n{summary}") 
    print('-'.join('' for x in range(100))) 
    print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")


In [ ]:
#MODEL GENERATED SUMMARY - ZERO SHOT
#prompt2 
sample_index=[20,60] 
for i,index in enumerate(sample_index): 
    dialogue=dataset['test'][index]['dialogue'] 
    summary=dataset['test'][index]['summary'] 
    prompt=f""" Summarize the following conversation. {dialogue} Summary: """ 
    inputs=tokenizer(prompt,return_tensors='pt') 
    output=tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True) 
    
    print('-'.join('' for x in range(100))) 
    print("Example ",i+1) 
    print('-'.join('' for x in range(100))) 
    print(f"INPUT PROMPT:\n{prompt}") 
    print('-'.join('' for x in range(100))) 
    print(f"BASELINE HUMAN SUMMARY:\n{summary}") 
    print('-'.join('' for x in range(100))) 
    print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")


In [ ]:
#MODEL GENERATED SUMMARY - ZERO SHOT
#Using prompt template: Template-1 
sample_index=[20,60] 
for i,index in enumerate(sample_index): 
    dialogue=dataset['test'][index]['dialogue'] 
    summary=dataset['test'][index]['summary'] 
    prompt=f""" Dialogue: {dialogue} What was going on? """  #template-1
    inputs=tokenizer(prompt,return_tensors='pt') 
    output=tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True) 

    print('-'.join('' for x in range(100))) 
    print("Example ",i+1) 
    print('-'.join('' for x in range(100))) 
    print(f"INPUT PROMPT:\n{prompt}") 
    print('-'.join('' for x in range(100))) 
    print(f"BASELINE HUMAN SUMMARY:\n{summary}") 
    print('-'.join('' for x in range(100))) 
    print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")


### ONE and FEW SHOT INFERENCE

In [ ]:
#One shot 
def make_prompt(example_indices_full,example_index_to_summarize): 
    prompt='' 
    for index in example_indices_full: 
        dialogue=dataset['test'][index]['dialogue'] 
        summary=dataset['test'][index]['summary'] 
        prompt+= f""" Dialogue: {dialogue} What was going on? {summary} """ 
        dialogue=dataset['test'][example_index_to_summarize]['dialogue'] 
        prompt+=f""" Dialogue: {dialogue} What was going on? """ 
        return prompt


In [ ]:
#building prompt
example_indices_full=[40] 
example_index_to_summarize=200 
one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize) 
print(one_shot_prompt)


In [ ]:
#generating summary:
summary=dataset['test'][example_index_to_summarize]['summary'] 
inputs=tokenizer(one_shot_prompt,return_tensors='pt') 
output=tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True) 
print('-'.join('' for x in range(100))) 
print(f"BASELINE HUMAN SUMMARY:\n{summary}\n") 
print('-'.join('' for x in range(100))) 
print(f"MODEL GENERATED SUMMARY - ONE SHOT:\n{output}")

In [ ]:
#Few Shot Inference 
#prompt building
example_indices_full = [40, 80, 120] 
example_index_to_summarize = 200 
few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize) 
print(few_shot_prompt)


In [ ]:
#generating summary:
summary = dataset['test'][example_index_to_summarize]['summary'] 
inputs = tokenizer(few_shot_prompt, return_tensors='pt') 
output = tokenizer.decode( model.generate( inputs["input_ids"], max_new_tokens=50, )[0], skip_special_tokens=True ) 
dash_line='-'.join('' for x in range(100)) 
print(dash_line) 
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n') 
print(dash_line) 
print(f'MODEL GENERATION - FEW SHOT:\n{output}')


### Generative Configuration Parameters for Inference

In [ ]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate( inputs["input_ids"], generation_config=generation_config, )[0], skip_special_tokens=True ) 
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')